In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from konlpy.tag import Okt
from gensim.models import FastText

In [4]:
df = pd.read_pickle("df_2030.pk")

In [5]:
df

,제목,본문,민원발생지,민원신청경로,접수기관,처리부서,성별,나이,접수일,year,type
0,로스쿨 특별전형 입학자 생계급여공제 관련청원,안녕하세요. 서울내 모대학교 법학전문대학원에 재학하고 있는 학생입니다. 법학전문대학...,서울특별시,국민신문고 Web,보건복지부,보건복지부 사회복지정책실 복지정책관 자립지원과,남자,36,2018-03-16 11:06:25,2018,일반
1,아파트 단지내에 장애인 주차구역,단지 내 주차구역도 신고가 되나요?\n\n다들 주차공간부족하고 눈치봐가면서 주차하고...,광주광역시,국민신문고 APP,광주광역시 북구,광주광역시 북구 복지경제국 노인장애인복지과,여자,34,2018-03-16 10:58:33,2018,일반
2,적법하지 못한 형태로 교습을 진행하고 있는 불법업체를 고발합니다.,[피민원인 정보] \r\n- 상호명 : 원주스파르타스터디센터독서실\r\n- 주소 :...,경기도,국민신문고 Web,강원도교육청,강원도교육청 강원도원주교육지원청 행정과,남자,38,2018-03-16 11:19:26,2018,일반
3,제품명 영문 혼용 표시 문의,귀 처의 무궁한 발전을 기원합니다. \r\n\r\n문의드리고자 함은 아래와 같습니다...,서울특별시,국민신문고 기관프레임 Web,식품의약품안전처,식품의약품안전처 식품안전정책국 식품안전표시인증과,남자,36,2018-03-16 14:20:20,2018,일반
4,온천천 도로 공사 관련 민원 신청의 건,안녕하세요 금정구에 거주중인 회사원입니다.\r\n\r\n요즘들어 상수도 공사가 한창...,부산광역시,국민신문고 Web,부산광역시,부산광역시 건설본부 토목시설부 토목2팀,남자,31,2018-03-16 11:42:54,2018,일반
...,...,...,...,...,...,...,...,...,...,...,...
4115,동탄 2신도시 경부고속도로 지하화 구간 남단 &#x28;800m&#x29; 차량 진...,안녕하세요.\r\n귀 기관의 노고에 감사드립니다. \r\n​\r\n동탄 2신도시 경...,경기도,국민신문고 Web,국민권익위원회,국민권익위원회 사무처 고충처리국 교통도로민원과,남자,36,2021-03-30 09:06:04,2021,고충
4116,배곧분동 똑바로 하십시오,1. 배곧 분동 공정하게 계획대로 7월에 처리해 주십시오. 몇몇 단체들의 욕심으로 ...,경기도,국민신문고 APP,국민권익위원회,국민권익위원회 사무처 고충처리국 행정문화교육민원과,여자,39,2021-03-30 14:58:45,2021,고충
4117,동탄2 경부지하화 구간 진출입로 방음터널 요청건,경부고속도로 직선화 구간 남단 &#x28;800M&#x29; 방음터널 추가 설치를 ...,경기도,국민신문고 Web,국민권익위원회,국민권익위원회 사무처 고충처리국 교통도로민원과,남자,38,2021-03-30 11:02:42,2021,고충
4118,사회복무요원은 죄를 지은 사람인가요?,저는 출퇴근이 왕복3시간 소요되는 사회복무요원입니다 .\r\n이로 인해 2주전부터 ...,경상북도,국민신문고 APP,국민권익위원회,국민권익위원회 사무처 고충처리국 국방보훈민원과,남자,21,2021-03-29 17:58:52,2021,고충


In [39]:
lst = []
for idx, txt in tqdm(enumerate(df['본문'])):
    try:
        if '출산' in txt or '임신' in txt:
            lst.append(idx)
    except:
        pass
len(lst)


2824205it [00:03, 829720.09it/s]


22816

In [42]:
df_birth = df.iloc[lst]
df_birth = df_birth.drop_duplicates('본문').reset_index(drop = True)
df_birth

,제목,본문,민원발생지,민원신청경로,접수기관,처리부서,성별,나이,접수일,year,type
0,"다산신도시 진건 2초, 유3의 중투위 통과를 요청합니다.","다산신도시는 정부가 지정한 보금자리 지구입니다. 양질의 교육과 편리한 교통, 쾌적한...",경기도,국민신문고 APP,교육부,교육부 교육복지정책국 지방교육재정과,여자,39,2018-03-30 09:24:00,2018,일반
1,"다산신도시 진건2초, 유3 중앙투자심사위원회 통과를 요청합니다",다산신도시 입주예정자로서 한말씀드리겠습니다. 현재 다산시도시는 5000세대가 입주한...,경기도,국민신문고 APP,교육부,교육부 교육복지정책국 지방교육재정과,여자,39,2018-03-30 09:16:54,2018,일반
2,"다산신도시 진건 2초, 유3의 중투위 통과를 요청합니다.",다산신도시 입주예정자입니다. \n도교육청과 교육부는 택지지구 탁상행정식 학교 신설 ...,경기도,국민신문고 APP,경기도교육청,경기도교육청 경기도구리남양주교육지원청 행정국 기획경영과,여자,39,2018-03-30 09:08:37,2018,일반
3,"다산 진건지구 진건2초, 유3의 중투위 통과시켜주세요.",신도시 특징상 아이들이 많은건 타신도시를봐도 아는거아닌지요.\n잘못된 전수조사로 진...,경기도,국민신문고 APP,교육부,교육부 교육복지정책국 지방교육재정과,여자,36,2018-03-30 09:42:30,2018,일반
4,"다산 진건 2초, 유3 중투위 통과 요청",다산신도시 입주예정자 입니다. \n진건지구는 보금자리지구로 실거주 분양자가 대부분이...,경기도,국민신문고 APP,교육부,교육부 교육복지정책국 지방교육재정과,여자,39,2018-03-30 10:23:36,2018,일반
...,...,...,...,...,...,...,...,...,...,...,...
12665,임산부 고충사항,안녕하세요 국군대전병원에 근무하는 안세현 주무관입니다. 두가지 애로사항이 있습니다...,대전광역시,국민신문고 모바일 Web,국민권익위원회,국민권익위원회 사무처 고충처리국 국방보훈민원과,여자,32,2020-11-04 16:17:28,2020,고충
12666,임신부를 경찰에 허위신고하는 난동을 부린 현직 군인,1. 안녕하세요 선생님. 늘 노고에 감사드립니다. \r\n2. 일전에 권익위에서 국...,서울특별시,국민신문고 Web,국민권익위원회,국민권익위원회 사무처 고충처리국 국방보훈민원과,여자,38,2020-11-17 11:20:51,2020,고충
12667,출산장려금 지급 거절,"현재 거주하고 있는 지역이 안산이고, 안산에서 거주기간이 8개월이 넘었습니다.\r\...",경기도,국민신문고 모바일 Web,국민권익위원회,국민권익위원회 사무처 고충처리국 복지노동민원과,여자,30,2020-11-23 09:51:08,2020,고충
12668,양육/육아수당 60일 지나 신청하여 소급적용 못받았습니다.,\r\n2020년 10월 29일 아들을 출산하였습니다. \r\n출생신고하면서 양육/...,서울특별시,국민신문고 Web,국민권익위원회,국민권익위원회 사무처 고충처리국 복지노동민원과,남자,39,2021-01-05 15:04:27,2021,고충


In [44]:
okt = Okt()
tok = []
for txt in tqdm(df_birth['본문']):
    try:
        if len(txt) < 3000:
            tok.append(okt.nouns(str(txt)))
        else:
            tok.append(["TOO" ,"LONG", "TXT"])
    except:
        tok.append(["NA"])

100%|██████████| 12670/12670 [20:00<00:00, 10.55it/s]


In [47]:
model = FastText(tok, window=6, min_count=5, workers=4, sg=1)

In [72]:
birth = []
birth_co_sim = []
pregnant = [] 
pregnant_co_sim = []
for word in model.wv.most_similar("출산", topn = 30):
    birth.append(word[0])
    birth_co_sim.append(word[1])

for word in model.wv.most_similar("임신", topn = 30):
    pregnant.append(word[0])
    pregnant_co_sim.append(word[1])



In [74]:
df_top30 = pd.DataFrame([birth, birth_co_sim, pregnant, pregnant_co_sim]).T

df_top30.columns = ['출산 연관단어', '출산과 코사인 유사도', '임신 연관단어', '임신과 코사인 유사도']
df_top30
df_top30.to_csv("2030_birth_top30.csv")